In [1]:
import sys
sys.path.insert(0, '../')
from escrape.commentary import CachedCommentary
import os

In [3]:
save_path = '../output/'
commentary = CachedCommentary(save_path, 'cricket', '10883', '1072308')

In [4]:
import funcy as fy
from escrape.commentary import ESPNParse

In [5]:
def ngrams(text, n=1):
    ngrams = []
    sub_toks = text.strip().split()
    l = len(sub_toks)
    for i in range(l-n+1):
        ngram = sub_toks[i:i+n]
        ngrams.append(tuple(ngram))
            
    return ngrams

In [6]:
from collections import Counter

counter = Counter(ngrams(text))

NameError: name 'text' is not defined

In [ ]:
from collections import defaultdict
def histogram(counter):
    occurrences = defaultdict(int)
    for key in counter:
        occurrences[counter[key]] += 1
    
    return occurrences

In [ ]:
xs, ys = zip(*sorted(histogram(counter).items(), key=lambda x: x[0]))

In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt

In [ ]:
print(xs, ys)

In [ ]:
oc = histogram(counter)
oc[6]

In [ ]:
points = []

for i in range(1, 50):
    total, unique = ngrams(text, i), set(ngrams(text, i))
    print(len(total), len(unique))
    points.append((i, len(unique)/len(total)))

xs, ys = list(zip(*points))

In [ ]:
plt.plot(xs, ys)
#plt.show()
plt.savefig('ngram_vs_fraction.png')

In [ ]:
from collections import Counter
points = []
for i in range(1, 50):
    total, unique = ngrams(text, i), set(ngrams(text, i))
    counter = Counter(total)
    print("-"*3, 'ngrams', i, '-'*3)
    most_frequent = fy.take(5, sorted(counter.items(), key=lambda x:x[-1], reverse=True))
    for key, value in most_frequent:
        print(' '.join(key), value)
    #print(len(total), len(unique))
    points.append((i, len(unique)/len(total)))

xs, ys = list(zip(*points))

In [7]:
import os
from escrape.season import get_season, get_match_nos
meta_path = os.path.join(save_path, 'meta.json')
# meta = open(meta_path, 'w+')

In [8]:
export = {}
for year in ['2017', '2016', '2015']:
    series, urls = get_season(year)
    matches = list(map(get_match_nos, urls))
    export[year] = list(zip(series, matches))    

KeyboardInterrupt: 

In [9]:
import json
export = json.load(open(meta_path))
for year in export:
    vy = export[year]
    for series, matches in vy:
        for match in matches:
            print(series, match)
            try:
                commentary = CachedCommentary(save_path, 'cricket', series, match)
                commentary.load()
                print("..Completed")
            except:
                print("..Failed")
                pass

FileNotFoundError: [Errno 2] No such file or directory: '../output/meta.json'

In [ ]:
files = os.listdir(save_path)
pkls = list(filter(lambda x: '.pkl' in x, files))
data = []
total_ngrams = set()
count = 0
n = 1
for i, pkl in enumerate(pkls):
    pkl = pkl[:-4]
    sport, series, match = pkl.split('_')
    commentary = CachedCommentary(save_path, sport, series, match)
    content = commentary.load()
    fcontent = ESPNParse(content)
    text = extract_raw_text(fcontent)
    pkl_ngrams = ngrams(text, 1)
    count += len(pkl_ngrams)
    total_ngrams = total_ngrams.union(set(pkl_ngrams))
    print(len(total_ngrams), count)
    data.append((len(total_ngrams), count))
    if i==100:
        break

In [ ]:
ys1, ys2 = list(zip(*data))
xs = list(range(1, len(ys1)+1))
from matplotlib import pyplot as plt
plt.clf()
plt.plot(xs, ys1, label='unique words')
#plt.plot(xs, ys2, label='total words')
#plt.plot(xs, ys2)

In [ ]:
plt.legend()
plt.savefig('unique-vs-iteration.png')
plt.show()

In [10]:
files = os.listdir(save_path)
pkls = list(filter(lambda x: '.pkl' in x, files))
data = []
total_ngrams = set()
count = 0
n = 1
for i, pkl in enumerate(pkls):
    pkl = pkl[:-4]
    sport, series, match = pkl.split('_')
    commentary = CachedCommentary(save_path, sport, series, match)
    
    datum = extract_key(ESPNParse(commentary.load()), 'text')
    data.append(datum)

In [9]:
def extract_key(commentary, key):   
    data = []
    for innings in commentary:
        for event in innings:
            data.append(event[key])
    return data

In [ ]:
def clean(datum):
    data = datum.split('\n')
    data = list(filter(lambda x: x, data))
    return data

lines = list(map(lambda x: map(clean, x), data))
lines = fy.flatten(lines)
non_empty_lines = list(filter(lambda x: x, lines))

In [ ]:
#print('\n'.join(fy.take(50, non_empty_lines)))


        


In [7]:
dump_path = '/tmp/jerin/machine-translation/data/cricket/parallel'
from ilmt.scrape.utils import takeWhileLength

In [ ]:
iterator = takeWhileLength(non_empty_lines, 5000)
if not os.path.exists(dump_path):
    os.mkdir(dump_path)
for i, x in enumerate(iterator):
    fpath = os.path.join(dump_path, str(i)+'.en')
    with open(fpath, 'w+') as fp:
        fp.write(x)

In [27]:
translated = list(filter(lambda x: '.hi' in x, os.listdir(dump_path)))
non_complete = list(filter(lambda x: 'complete' not in x, translated))

In [28]:
_src = open(os.path.join(dump_path, 'complete.en'), 'w+')
_tgt = open(os.path.join(dump_path, 'complete.hi'), 'w+')
for hi_file in non_complete:
    src = hi_file.replace('.hi', '.en')
    tgt = hi_file
    print(src, tgt)
    src = os.path.join(dump_path, src)
    tgt = os.path.join(dump_path, tgt)
    for en_line, hi_line in zip(open(src), open(tgt)):
        print(en_line.strip(), file=_src)
        print(hi_line.strip(), file=_tgt)

95.en 95.hi
92.en 92.hi
71.en 71.hi
34.en 34.hi
75.en 75.hi
43.en 43.hi
74.en 74.hi
86.en 86.hi
68.en 68.hi
38.en 38.hi
63.en 63.hi
27.en 27.hi
40.en 40.hi
60.en 60.hi
65.en 65.hi
58.en 58.hi
61.en 61.hi
9.en 9.hi
4.en 4.hi
94.en 94.hi
2.en 2.hi
76.en 76.hi
55.en 55.hi
6.en 6.hi
5.en 5.hi
98.en 98.hi
99.en 99.hi
35.en 35.hi
48.en 48.hi
73.en 73.hi
50.en 50.hi
88.en 88.hi
47.en 47.hi
69.en 69.hi
51.en 51.hi
66.en 66.hi
70.en 70.hi
62.en 62.hi
28.en 28.hi
67.en 67.hi
77.en 77.hi
57.en 57.hi
82.en 82.hi
36.en 36.hi
29.en 29.hi
39.en 39.hi
59.en 59.hi
7.en 7.hi
84.en 84.hi
96.en 96.hi
80.en 80.hi
87.en 87.hi
79.en 79.hi
37.en 37.hi
54.en 54.hi
33.en 33.hi
78.en 78.hi
91.en 91.hi
85.en 85.hi
1.en 1.hi
44.en 44.hi
41.en 41.hi
83.en 83.hi
53.en 53.hi
42.en 42.hi
46.en 46.hi
64.en 64.hi
93.en 93.hi
45.en 45.hi
32.en 32.hi
31.en 31.hi
52.en 52.hi
97.en 97.hi
81.en 81.hi
72.en 72.hi
0.en 0.hi
30.en 30.hi
56.en 56.hi
89.en 89.hi
8.en 8.hi
90.en 90.hi
49.en 49.hi


In [29]:
from ilmt.preproc.split import prepare

In [30]:
pre = os.path.join(dump_path, 'complete')
prepare(prefix=pre, source='en', target='hi', train=0.4, test=0.4, dev=0.2, randomize=True)

train 2387
dev 1194
test 2388
